In [37]:
#Projet de Sihem Yenbou et de Güney Naz Tümüklü

In [ ]:
pip install --user tensorflow==2.15.0 numpy==1.23.5 pandas==1.5.3 matplotlib==3.7.1

In [1]:
#Import 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [7]:
#Analyse descriptives des données
energydata = pd.read_csv("energydata_complete.csv")
print(energydata)

                      date  Appliances  lights         T1       RH_1  \
0      2016-01-11 17:00:00          60      30  19.890000  47.596667   
1      2016-01-11 17:10:00          60      30  19.890000  46.693333   
2      2016-01-11 17:20:00          50      30  19.890000  46.300000   
3      2016-01-11 17:30:00          50      40  19.890000  46.066667   
4      2016-01-11 17:40:00          60      40  19.890000  46.333333   
...                    ...         ...     ...        ...        ...   
19730  2016-05-27 17:20:00         100       0  25.566667  46.560000   
19731  2016-05-27 17:30:00          90       0  25.500000  46.500000   
19732  2016-05-27 17:40:00         270      10  25.500000  46.596667   
19733  2016-05-27 17:50:00         420      10  25.500000  46.990000   
19734  2016-05-27 18:00:00         430      10  25.500000  46.600000   

              T2       RH_2         T3       RH_3         T4  ...         T9  \
0      19.200000  44.790000  19.790000  44.730000  19.0

In [9]:
energydata.size

572315

In [7]:
#Nous avons une base de données qui comportent 19735 lignes pour 29 colonnes. Cela donne un total de 572315 données.
#Les colonnes réprensentent les variables (température, humidité...).
#Nous avons donc 29 variables :
#Les variables Ti, i allant de 1 à 9, représentent la température mesurée de plusieurs pièces d'un bâtiment et la température extérieure (To et T6).
#Les variables RH_i, i allant de 1 à 9, représentent l'humidité mesurée de plusieurs pièces d'un bâtiment et l'humidité extérieure (RH_out et RH_6).
#Il y a également des variables représentant d'autres facteurs météorologiques (vitesse du vent, pression, point de rosée...).
#Enfin, nous avons 2 variables aléatoires rv1 et rv2 qui nous permettront de tester le modèle de régression choisi et implémenté.

In [9]:
#La colonne Appliances représente l'énergie utilisée en Wh (Watt par heure), c'est notre cible. 

In [11]:
#Les lignes représentent la date et l'heure à laquelle la température et l'humidité de chaque pièce/endroit a été mesuré.
#Ces mesures ont été collectées toutes les 10 minutes pendant environ 4,5 mois.

In [11]:
#Séparation des données en données d'entraînement et de validation pour la comparaison des méthodes
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(energydata.iloc[:,2:])

np.shape(X)

(19735, 27)

In [13]:
#Données d'entraînement
nbdatatrain = 15000
X_train = X[:nbdatatrain,:] 
Y_train = energydata.iloc[:nbdatatrain,1] 
np.shape(X_train)

(15000, 27)

In [15]:
np.shape(Y_train)

(15000,)

In [17]:
#Données de validation
X_test  = X[nbdatatrain:,:] 
Y_test  = energydata.iloc[nbdatatrain:,1] 
np.shape(X_test)

(4735, 27)

In [19]:
np.shape(Y_test)

(4735,)

In [ ]:
#Présentation de la régression Lasso avec pénalisation et des hyperparamètres dont elle dépend

## Régression Lasso avec pénalisation

Dans ce projet, nous cherchons à prédire la consommation d’énergie des appareils ménagers (colonne `Appliances`, exprimée en Wh) à partir de variables environnementales et électriques mesurées dans un foyer.

La régression Lasso (Least Absolute Shrinkage and Selection Operator) est une méthode de régression linéaire pénalisée, qui permet à la fois d’améliorer la performance du modèle en réduisant le surapprentissage et de réaliser une sélection automatique de variables en annulant certains coefficients.

Le modèle s’écrit sous la forme suivante :
$
Y = X \beta + \varepsilon,
$
où :
- ($ Y \in \mathbb{R}^n $) représente la consommation d’énergie (notre variable cible),
- ($ X \in \mathbb{R}^{n \times d} $) est la matrice des variables explicatives (température, humidité, tension, intensité, etc.),
- ($ \beta \in \mathbb{R}^d $) est le vecteur des coefficients à estimer,
- ($ \varepsilon $) est le terme d’erreur.

Ici n = 15000 et d = 27.

L’estimateur Lasso consiste à minimiser la fonction de coût suivante :
$
\widehat{\beta} = \underset{\beta \in \mathbb{R}^d}{\arg\min} \left( \frac{1}{n} \| Y - X\beta \|_2^2 + \lambda \| \beta \|_1 \right),
$
où :
- $\| Y - X\beta \|_2^2 $ est l’erreur quadratique moyenne (MSE),
- $\| \beta \|_1 $ est la norme L1 (somme des valeurs absolues des coefficients),
- $\lambda > 0 $ est un **hyperparamètre** qui contrôle l’intensité de la pénalisation.

### Choix de l’hyperparamètre $\lambda $

- Si $\lambda $ est trop petit : le modèle est peu régularisé, risque de surapprentissage.
- Si $\lambda $ est trop grand : trop de coefficients sont annulés, perte d’information.

Pour choisir une valeur optimale de $\lambda$, nous utilisons la **validation croisée**. Cela permet de tester différentes valeurs de $\lambda$ et de sélectionner celle qui minimise l’erreur de prédiction sur des données non vues (`X_test`et `Y_test`).

In [ ]:
#Présentation de la régression Lasso sans pénalisation et des hyperparamètres dont elle dépend

## Régression linéaire sans pénalisation

Le modèle de régression linéaire classique (sans pénalisation) s’écrit sous la forme :

$
Y = X \beta + \varepsilon
$

où :

- ($ Y \in \mathbb{R}^n $) représente la consommation d’énergie (notre variable cible),
- ($ X \in \mathbb{R}^{n \times d} $) est la matrice des variables explicatives (température, humidité, tension, intensité, etc.),
- ($ \beta \in \mathbb{R}^d $) est le vecteur des coefficients à estimer,
- ($ \varepsilon $) est le terme d’erreur.

L’estimation de ($ \beta $) est obtenue en minimisant l’erreur quadratique moyenne (MSE) :

$$
\widehat{\beta} = \underset{\beta \in \mathbb{R}^d}{\arg\min} \, \frac{1}{n} \|Y - X\beta\|_2^2
$$

Cette méthode ne dépend pas d'hyperparamètre.

In [ ]:
#Discussion sur les choix d'implémentation et l'influence des différents paramètres sur les résultats

In [ ]:
#Implémentation avec réseaux de neurones denses

In [23]:
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV

In [ ]:
#Analyse des performances et des pistes d'amélioration possibles